<a href="https://colab.research.google.com/github/jben-hun/colab_notebooks/blob/master/algorithms/markov_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation

In [1]:
!pip install -q praw

import praw
import re
import random
import tqdm
import numpy as np
import pandas as pd
from collections import defaultdict
from collections import deque

pd.set_option("max_colwidth", None)

client_id = "" #@param {type:"string"}
client_secret = "" #@param {type:"string"}
user_agent = "" #@param {type:"string"}

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent)

     |████████████████████████████████| 153kB 4.8MB/s 
     |████████████████████████████████| 204kB 7.9MB/s 


In [2]:
SUBREDDITS = ("askreddit", "explainlikeimfive", "dankmemes")


class RedditMarkovChain:
    def __init__(
            self,
            subreddit,
            sentence_limit=1000,
            begin_str="*BEGIN*",
            end_str="*END*",
            cycle_str="*CYCLE*",
            train_split=(0.9)):
        self.__subreddit = subreddit
        self.__sentence_limit = sentence_limit
        self.__begin_str = begin_str
        self.__end_str = end_str
        self.__cycle_str = cycle_str
        self.__train_split = train_split
        self.__test_split = (1.0 - train_split)

        self.__sentences = self.mine_subreddit(
            subreddit=reddit.subreddit(self.subreddit),
            sentence_limit=self.sentence_limit)

        self.model = self.__build_model()

    def __build_model(self):
        """Build a markov chain model from extracted sentences"""
        model = defaultdict(lambda: defaultdict(lambda: 0))

        for sentence in self.train_sentences:
            words = self.split_sentence(sentence)
            model[self.begin_str][words[0]] += 1
            model[words[-1]][self.end_str] += 1
            for i in range(len(words) - 1):
                model[words[i]][words[i + 1]] += 1

        return model

    def generate(self, method="sample"):
        """Generate text using the created markov chain model

        method:
            expected: choose most likely words, infinite cycles are possible
            random: choose words uniformly
            sample: choose words based on the modeled probabilities
        """

        sentence = ""
        word = self.begin_str

        if method == "expected":
            used = set()

        while True:
            if method == "expected":
                word = max(
                    self.model[word].items(), key=lambda x: x[1])[0]
            elif method == "random":
                word = random.choice(tuple(self.model[word].items()))[0]
            elif method == "sample":
                words = tuple(self.model[word].keys())
                probs = self.get_probs(self.model[word])
                word = np.random.choice(words, p=probs)
            if word == self.end_str:
                break
            if word not in ".?!,":
                sentence += " "
            sentence += word

            if method == "expected":
                if word in used:
                    sentence += f" {self.cycle_str}"
                    break
                used.add(word)

        return sentence

    def classify(self, sentence):
        """Deduce the most likely source of a sentence"""

        words = self.split_sentence(sentence)

        p = self.get_prob(self.model[self.begin_str], words[0])
        for i in range(len(words)-1):
            p *= self.get_prob(self.model[words[i]], words[i+1])
        p *= self.get_prob(self.model[words[-1]], self.end_str)

        return p

    @classmethod
    def mine_subreddit(cls, subreddit, sentence_limit):
        """Extract clean sentences from submissions and comments"""

        # re that matches clean sentences
        matcher = re.compile(r"(?:[.!?] |^)[A-Z][\w', ]+[.!?](?= [A-Z]|$)")

        sentences = []

        with tqdm.tqdm(total=sentence_limit) as pbar:
            for submission in subreddit.hot(limit=None):
                sentences += matcher.findall(submission.title)
                sentences += matcher.findall(submission.selftext)

                submission.comment_sort = "best"

                comments = [
                    comment.body for comment in submission.comments.list()
                    if not isinstance(comment, praw.models.MoreComments)]

                for comment in comments:
                    sentences += matcher.findall(comment)

                len_sentences = len(sentences)
                if len_sentences >= sentence_limit:
                    random.shuffle(sentences)
                    pbar.update(sentence_limit - pbar.n)
                    break
                else:
                    pbar.update(len_sentences - pbar.n)

        return [cls.process_sentence(sentence) for sentence
                in sentences[:sentence_limit]]

    @staticmethod
    def process_sentence(sentence):
        """Clean up sentences"""
        return (sentence.lstrip(".!? ")
                        .replace("won't", "will not")
                        .replace("n't", " not")
                        .replace("'m", " am")
                        .replace("'re", " are"))

    @staticmethod
    def split_sentence(sentence):
        """Split sentences into words"""
        return re.findall(r"((?:[\w']+)|(?:[,!.?]))", sentence)

    @staticmethod
    def get_prob(d, word):
        """Get single probability from word counts"""
        return 0 if word not in d else d[word]/sum(d.values())

    @staticmethod
    def get_probs(d):
        """Get all probabilities from word counts"""
        n = sum(d.values())
        return [v/n for v in d.values()]

    @staticmethod
    def traverse_comments(comments, *, breadth_first=False):
        queue = deque(comments[:])
        result = []
        while queue:
            e = queue.pop()
            if isinstance(e, praw.models.MoreComments):
                if breadth_first:
                    queue.extendleft(e.comments())
                else:
                    queue.extend(e.comments())
            else:
                if breadth_first:
                    queue.extendleft(e.replies)
                else:
                    queue.extend(e.replies)
                result.append(e)
        return result

    @property
    def subreddit(self):
        return self.__subreddit

    @property
    def sentences(self):
        return self.__sentences

    @property
    def sentence_limit(self):
        return self.__sentence_limit

    @property
    def begin_str(self):
        return self.__begin_str

    @property
    def end_str(self):
        return self.__end_str

    @property
    def cycle_str(self):
        return self.__cycle_str

    @property
    def train_split(self):
        return self.__train_split

    @property
    def test_split(self):
        return self.__test_split

    @property
    def train_sentences(self):
        return self.sentences[:int(len(self.sentences)*self.train_split)]

    @property
    def test_sentences(self):
        return self.sentences[int(len(self.sentences)*self.train_split):]

# Demo

In [3]:
chains = {subreddit: RedditMarkovChain(subreddit, sentence_limit=1000)
          for subreddit in SUBREDDITS}

100%|██████████| 1000/1000 [00:32<00:00, 30.90it/s]


**Deriving most probable sentence for each model**

In [4]:
for subreddit, chain in chains.items():
    print(f"{subreddit}: {chain.generate('expected')}")

askreddit:  I was a kid.
explainlikeimfive:  I think it's not a lot of the same with the *CYCLE*
dankmemes:  I am not the imposter.


**Generating new text**

In [5]:
dict_data = defaultdict(lambda: [])

for subreddit, chain in chains.items():
    for _ in range(5):
        sentence = chain.generate()
        dict_data["sentence"].append(sentence)
        dict_data["model"].append(subreddit)

        for k, v in chains.items():
            p = v.classify(sentence)
            dict_data[f"P({k})"].append(p)

display(pd.DataFrame(dict_data))

,sentence,model,P(askreddit),P(explainlikeimfive),P(dankmemes)
0,"They could be a year, his entire tenure on by the battle with the Justice, I believed.",askreddit,8.500275e-26,0.000000e+00,0.000000e+00
1,One does not stop calling to it is a kid.,askreddit,4.529957e-13,0.000000e+00,0.000000e+00
2,"I mean, Roe v Casey is such a liberal.",askreddit,4.942273e-14,0.000000e+00,0.000000e+00
3,"Yup, I am not a few weeks into my high school sent to get representation in my age.",askreddit,3.556284e-22,0.000000e+00,0.000000e+00
4,One does the presidency so often.,askreddit,2.909887e-09,0.000000e+00,0.000000e+00
5,Can you.,explainlikeimfive,9.132420e-05,9.876543e-05,1.769912e-04
6,So kernel itself is pretty simple explanation friend.,explainlikeimfive,0.000000e+00,5.293123e-10,0.000000e+00
7,A 16 year old human babies I think it's making the game.,explainlikeimfive,0.000000e+00,8.364851e-13,0.000000e+00
8,And the right order to being in momentum.,explainlikeimfive,0.000000e+00,1.239369e-12,0.000000e+00
9,This is easier target.,explainlikeimfive,0.000000e+00,1.089325e-05,0.000000e+00


**Classifying real text**

In [6]:
dict_data = defaultdict(lambda: [])

for subreddit, chain in chains.items():
    for sentence in chain.test_sentences[:5]:
        dict_data["sentence"].append(sentence)
        dict_data["source"].append(subreddit)

        for k, v in chains.items():
            p = v.classify(sentence)
            dict_data[f"P({k})"].append(p)

display(pd.DataFrame(dict_data))

,sentence,source,P(askreddit),P(explainlikeimfive),P(dankmemes)
0,I think the death of Artax is still one of the most heartbreaking moments in movie history!,askreddit,0.0,0.0,0.000000
1,I'd still call it a beautiful friendship.,askreddit,0.0,0.0,0.000000
2,Cool Runnings.,askreddit,0.0,0.0,0.000000
3,We are fucked even if he is not.,askreddit,0.0,0.0,0.000000
4,I always get the feeling that people who post this type of stuff are lying just to get attention and karma.,askreddit,0.0,0.0,0.000000
5,"You are correct, a wider base of support is more stable.",explainlikeimfive,0.0,0.0,0.000000
6,You should check it out!,explainlikeimfive,0.0,0.0,0.000000
7,"Gaseous compounds naturally want to fill the area they are in equally, so that each molecule is an equal distance away from each other molecule.",explainlikeimfive,0.0,0.0,0.000000
8,I have had pets that I do not immeditaely love.,explainlikeimfive,0.0,0.0,0.000000
9,"Some people, very much do, have an emotional connection with their car.",explainlikeimfive,0.0,0.0,0.000000


# TODO

*   Second order markov chains: P(AB->C)

# References

*   https://en.wikipedia.org/wiki/Markov_chain
*   https://www.reddit.com/r/SubredditSimulator/comments/3g9ioz/what_is_rsubredditsimulator/
*   https://www.reddit.com/r/SubSimulatorGPT2/comments/btfhks/what_is_rsubsimulatorgpt2/